# Deploying NVIDIA Triton Inference Server in AI Platform Prediction Custom Container

In this notebook, we will walk through the process of deploying NVIDIA's Triton Inference Server into AI Platform Prediction Custom Container service in the Direct Model Server mode:

![](img/caip_triton_container_diagram_direct.jpg)


In [ ]:
%env PROJECT_ID="[enter project name]"
%env MODEL_BUCKET="gs://[enter GCS bucket name]"
%env ENDPOINT="https://alpha-ml.googleapis.com/v1"


In [ ]:
%env PROJECT_ID=tsaikevin-triton-2
%env MODEL_BUCKET=gs://tsaikevin-triton-2-models
%env ENDPOINT=https://alpha-ml.googleapis.com/v1


### Prepare model Artifacts

Clone the NVIDIA Triton Inference Server repo.

In [ ]:
!git clone https://github.com/NVIDIA/triton-inference-server.git

Create the GCS bucket where the model artifacts will be copied to.

In [ ]:
!gsutil mb $MODEL_BUCKET

Stage model artifacts and copy to bucket.

In [ ]:
!mkdir model_repository

In [ ]:
!cp -R triton-inference-server/docs/examples/model_repository/* model_repository/

In [ ]:
!./triton-inference-server/docs/examples/fetch_models.sh

In [ ]:
!gsutil -m cp -R model_repository/ $MODEL_BUCKET

In [ ]:
!gsutil ls $MODEL_BUCKET/model_repository

## Create and deploy Model and Model Version

In this section, we will deploy two models:
1. Simple model with non-binary data.  KF Serving v2 protocol specifies a json format with non-binary data in the json body itself.
2. Binary data model with ResNet-50.  Triton's implementation of binary data for KF Server v2 protocol.

### Simple Model with non-binary data

#### Create Model

AI Platform Prediction uses a Model/Model Version Hierarchy, where the Model is a logical grouping of Model Versions.  We will first create the Model.

Because the MODEL_NAME variable will be used later to specify the predict route, and Triton will use that route to run prediction on a specific model, we must set the value of this variable to a valid name of a model.  For this section, will use the "simple" model.

In [ ]:
%env MODEL_NAME=simple

In [ ]:
!curl --request \
    -X POST -v -k -H "Content-Type: application/json" \
    -d "{'name': '"$MODEL_NAME"'}" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_ID}/models/"

#### Create Model Version

After the Model is created, we can now create a Model Version under this Model.  Each Model Version will need a name that is unique within the Model.  In AI Platform Prediction Custom Container, a {Project}/{Model}/{ModelVersion} uniquely identifies the specific container and model artifact used for inference.

In [ ]:
%env VERSION_NAME=v20

The following specifications tell AI Platform how to create the Model Version.

In [ ]:
import json
import os

triton_simple_version = {
  "name": os.getenv("VERSION_NAME"),
  "deployment_uri": os.getenv("MODEL_BUCKET")+"/model_repository",
  "container": {
    "image": "gcr.io/"+os.getenv("PROJECT_ID")+"/tritonserver:20.06-py3",
    "args": ["tritonserver",
             "--model-repository=$(AIP_STORAGE_URI)"
    ],
    "env": [
    ], 
    "ports": [
      { "containerPort": 8000 }
    ]
  },
  "routes": {
    "predict": "/v2/models/"+os.getenv("MODEL_NAME")+"/infer",
    "health": "/v2/models/"+os.getenv("MODEL_NAME")
  },
  "machine_type": "n1-standard-4",
  "acceleratorConfig": {
    "count":1,
    "type":"nvidia-tesla-t4"
  }
}

with open("triton_simple_version.json", "w") as f: 
  json.dump(triton_simple_version, f)

In [ ]:
triton_simple_version

In [ ]:
!curl --request \
    POST -v -k -H "Content-Type: application/json" \
    -d @triton_simple_version.json \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_ID}/models/${MODEL_NAME}/versions"

#### Check the status of Model Version creation

Creating a Model Version may take several minutes.  You can check on the status of this specfic Model Version with the following, and a successful deployment will show:

`"state": "READY"`

In [ ]:
!curl --request GET -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_ID}/models/${MODEL_NAME}/versions/${VERSION_NAME}" 

#### To list all Model Versions and their states in this Model:

In [ ]:
!curl --request GET -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_ID}/models/${MODEL_NAME}/versions/" 

#### Run Prediction

The "simple" model takes two tensors with shape [1,16] and does a couple of basic arithmetic operation.

In [ ]:
!curl -X POST ${ENDPOINT}/projects/${PROJECT_ID}/models/${MODEL_NAME}/versions/${VERSION_NAME}:predict \
    -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    -d '{ \
            "id": "0", \
            "inputs": [ \
                { \
                    "name": "INPUT0", \
                    "shape": [1, 16], \
                    "datatype": "INT32", \
                    "parameters": {}, \
                    "data": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] \
                }, \
                { \
                    "name": "INPUT1", \
                    "shape": [1, 16], \
                    "datatype": "INT32", \
                    "parameters": {}, \
                    "data": [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1] \
                } \
            ] \
        }'

### ResNet-50 with binary data

#### Create Model

In [ ]:
%env BINARY_MODEL_NAME=resnet50_netdef

In [ ]:
!curl -X POST -v -k -H "Content-Type: application/json" \
  -d "{'name': '"$BINARY_MODEL_NAME"'}" \
  -H "Authorization: Bearer `gcloud auth print-access-token`" \
  "${ENDPOINT}/projects/${PROJECT_ID}/models/"

#### Create Model Version

In [ ]:
%env BINARY_VERSION_NAME=v2

In [ ]:
triton_binary_version = {
  "name": os.getenv("BINARY_VERSION_NAME"),
  "deployment_uri": os.getenv("MODEL_BUCKET")+"/model_repository",
  "container": {
    "image": "gcr.io/"+os.getenv("PROJECT_ID")+"/tritonserver:20.06-py3",
    "args": ["tritonserver",
             "--model-repository=$(AIP_STORAGE_URI)"
    ],
    "env": [
    ], 
    "ports": [
      { "containerPort": 8000 }
    ]
  },
  "routes": {
    "predict": "/v2/models/"+os.getenv("BINARY_MODEL_NAME")+"/infer",
    "health": "/v2/models/"+os.getenv("BINARY_MODEL_NAME")
  },
  "machine_type": "n1-standard-4",
  "acceleratorConfig": {
    "count":1,
    "type":"nvidia-tesla-t4"
  }
}

with open("triton_binary_version.json", "w") as f: 
  json.dump(triton_binary_version, f)

In [ ]:
!curl -X POST -v -k -H "Content-Type: application/json" \
  -d @triton_binary_version.json \
  -H "Authorization: Bearer `gcloud auth print-access-token`" \
  ${ENDPOINT}/projects/${PROJECT_ID}/models/${BINARY_MODEL_NAME}/versions

#### Check Model Version status

In [ ]:
!curl --request GET -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "${ENDPOINT}/projects/${PROJECT_ID}/models/${BINARY_MODEL_NAME}/versions/${BINARY_VERSION_NAME}" 

#### Prepare Binary Request Payload

Triton's implementation of KF Serving v2 protocol for binary data appends the binary data after the json body.  Triton requires an additional header for offset:

`Inference-Header-Content-Length: [offset]`

We have provided a script that will automatically resize the image to the proper size for ResNet-50 [224, 224, 3] and calculate the proper offset.

In [ ]:
!pip3 install geventhttpclient

The following command takes an image file and outputs the necessary data structure for Triton.

In [ ]:
!python3 get_request_body_simple.py -m image -f triton-inference-server/qa/images/mug.jpg

#### Run Prediction

In [ ]:
!curl --request POST ${ENDPOINT}/projects/${PROJECT_ID}/models/${BINARY_MODEL_NAME}/versions/${BINARY_VERSION_NAME}:predict \
    -k -H "Content-Type: application/octet-stream" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    -H "Inference-Header-Content-Length: 138" \
    --data-binary @payload.dat